# Практическое задание 2 (часть 2)

# Распознавание именованных сущностей из Twitter с помощью LSTM

## курс "Математические методы анализа текстов"


### ФИО: Хуршудов Артем Эрнестович

## Введение

### Постановка задачи

В этом задании вы будете использовать рекуррентные нейронные сети для решения проблемы распознавания именованных сущностей (NER). Примерами именованных сущностей являются имена людей, названия организаций, адреса и т.д. В этом задании вы будете работать с данными twitter.

Например, вы хотите извлечь имена и названия организаций. Тогда для текста

    Yan Goodfellow works for Google Brain

модель должна извлечь следующую последовательность:

    B-PER I-PER    O     O   B-ORG  I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система введена, чтобы различать последовательные именованные сущности одного типа.

Решение этого задания будет основано на нейронных сетях, а именно на Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Numpy](http://www.numpy.org).
 
### Данные

Все данные содержатся в папке `./data`: `./data/train.txt`, `./data/validation.txt`, `./data/test.txt`.

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1s1rFOFMZTBqtJuQDcIvW-8djA78iUDcx)

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Часть 1. Подготовка данных (2 балла)

### Загрузка данных

Мы будем работать с данными, которые содержат твиты с тегами именованных сущностей. Каждая строка файла содержит пару токен(слово или пунктуация) и тег, разделенные пробелом. Различные твиты разделены пустой строкой.

Функция *read_data* считывает корпус из *file_path* и возвращает два списка: один с токенами и один с соответствующими токенам тегами. Также она заменяет все ники (токены, которые начинаются на символ *@*) на токен `<USR>` и url-ы (токены, которые начинаются на *http://* или *https://*) на токен `<URL>`. 

Вам необходимо реализовать эту функцию.

In [0]:
def read_data(file_path):
    with open(file_path, 'r') as f:
      lines = f.readlines()
    arr = []
    tmp = []
    for el in lines:
      if el != '\n':
        tmp.append(el)
      else:
        arr.append(tmp)
        tmp = []
    tokens = [[el.split()[0] for el in sent] 
              for sent in arr]
    tags = [[el.split()[1] for el in sent] 
              for sent in arr]

    for sent_idx in range(len(tokens)):
      for token_idx in range(len(tokens[sent_idx])):
        if tokens[sent_idx][token_idx][0] == '@':
          tags[sent_idx][token_idx] = '<USR>'
        if (tokens[sent_idx][token_idx][:7] == 'http://') or \
           (tokens[sent_idx][token_idx][:8] == 'https://'):
          tags[sent_idx][token_idx] = '<URL>'
    
    return tokens, tags

Теперь мы можем загрузить 3 части данных:
 - *train* для тренировки модели;
 - *validation* для валидации и подбора гиперпараметров;
 - *test* для финального тестирования.

In [0]:
_dir = 'gdrive/My Drive/mmta/RNN/'

train_tokens, train_tags = read_data(_dir + 'data/train.txt')
validation_tokens, validation_tags = read_data(_dir + 'data/validation.txt')
test_tokens, test_tags = read_data(_dir + 'data/test.txt')

Всегда полезно знать, с какими данными вы работаете. Выведем небольшую часть.

In [5]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
@TheValarium	<USR>
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
http://t.co/2zgQ99nmuf	<URL>
http://t.co/eQmogqqABK	<URL>

Happy	O
Birthday	O
@AshForeverAshey	<USR>
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения.

- {token}$\to${token id}: устанавливает соответствие между токеном и строкой в embedding матрице;
- {tag}$\to${tag id}: one hot encoding тегов.


Теперь вам необходимо реализовать функцию *build_dict*, которая должна возвращать словарь {token or tag}$\to${index} и контейнер, задающий обратное отображение.

In [0]:
from collections import defaultdict

In [0]:
def build_dict(tokens_or_tags, special_tokens):
    """
    tokens_or_tags: a list of lists of tokens or tags
    special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens to indices and vice versa
    # Add special tokens to dictionaries
    # The first special token must have index 0

    for idx, el in enumerate(special_tokens):
      tok2idx[el] = idx
      idx2tok.append(el)
      
    all_tokens = set([el 
                  for sent in tokens_or_tags
                  for el in sent])
    
    for el in special_tokens:
      if el in all_tokens:
        all_tokens.remove(el)
    
    for idx, el in enumerate(all_tokens):
      tok2idx[el] = idx + len(special_tokens)
      idx2tok.append(el)
    
    return tok2idx, idx2tok

После реализации функции *build_dict* вы можете создать словари для токенов и тегов. В нашем случае специальными токенами будут:
 - `<UNK>` токен для обозначаения слов, которых нет в словаре;
 - `<PAD>` токен для дополнения предложений одного батча до одинаковой длины.

In [0]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

### Генератор батчей

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины. Для некоторых фреймворков (таких как tensorflow) это необходимо сделать до подачи батча в нейронную сеть. В случае с pytorch это можно сделать как вне архитектуры нейронной сети, так и внутри. Мы выбрали более универсальный вариант и наш генератор батчей дополняет все последовательности внутри одного батча до одной длины.

Генератор батчей разбивает последовательность входных предложений и тегов на батчи размера batch_size. Размер последнего батча может быть меньше, если allow_smaller_last_batch is True, иначе последний батч исключается из генератора. Если включён параметр shuffle, данные перед разделением на батчи будут перемешаны. 

In [0]:
import random

def batches_generator(batch_size, tokens_idxs, tags_idxs,
                      shuffle=True, allow_smaller_last_batch=True, device='cpu'):
    """
    Generates padded batches of tags_idxs and tags_idxs.
    
    batch_size : int, number of objects in one batch
    tokens_idxs : list of list of int
    tags_idxs : list of list of int
    shuffle : bool
    allow_smaller_last_batch : bool
    device: str, cpu or cuda:x
    
    yield x, y: torch.LongTensor and torch.LongTensor
    x - batch of tokens_idxs, y - batch of tags_idxs
    """
    n_samples = len(tokens_idxs)
    
    tokens_idxs = np.array(tokens_idxs)
    tags_idxs = np.array(tags_idxs)
    
    if shuffle:
      idxs = list(range(len(tokens_idxs)))
      random.shuffle(idxs)
      tokens_idxs = tokens_idxs[idxs]
      tags_idxs = tags_idxs[idxs]
    
    # Get the number of batches
    n_batches = int(n_samples / batch_size)
    if n_samples%batch_size: 
      n_batches += allow_smaller_last_batch
      
    
    # For each k yield pair x and y
    for k in range(n_batches):
        cur_sent = tokens_idxs[k*batch_size : (k+1)*batch_size]
        cur_tags = tags_idxs[k*batch_size : (k+1)*batch_size]
        sent_len = max([len(el) for el in cur_sent])

        for i in range(len(cur_sent)):
          while len(cur_sent[i]) < sent_len:
            cur_sent[i].append(token2idx['<PAD>'])
            cur_tags[i].append(tag2idx['O'])
        
        cur_sent = np.array([np.array(el) for el in cur_sent])
        cur_tags = np.array([np.array(el) for el in cur_tags])
        
        
        x = torch.LongTensor(cur_sent)
        y = torch.LongTensor(cur_tags)
        
        x = x.long().to(device)
        y = y.long().to(device)
        
        yield x, y

Протестируйте ваш генератор батчей:

In [0]:
train_idxs = [[token2idx[el] for el in sent] for sent in train_tokens]
train_tag_idxs = [[tag2idx[el] for el in sent] for sent in train_tags]

test_idxs = [[token2idx[el] for el in sent] for sent in test_tokens]
test_tag_idxs = [[tag2idx[el] for el in sent] for sent in test_tags]

In [11]:
np.array(train_idxs)

array([list([2934, 26243, 16661, 1146, 19044, 7027, 23503, 3304, 13647, 19264, 22757, 10796, 20368, 4271, 8397, 23272, 14165, 25910, 20958, 24657, 17738, 23675, 12604, 489, 9173, 22350]),
       list([16245, 5719, 19357, 10098, 21602, 5217, 22662, 4105, 9710, 25407, 13794, 9746, 5355, 1052, 4105, 14743, 11474, 15191, 10722, 25714, 13434]),
       list([19841, 24276, 19040, 2413, 17909, 14321, 10237, 17920, 424, 17712, 21366, 8247, 21368, 20958]),
       ...,
       list([23248, 24580, 4018, 20825, 5631, 15603, 4478, 9314, 24785, 19167, 10521, 16030, 3554, 920, 13150, 15248, 18606, 10086, 15638, 19980]),
       list([9551, 10183, 22462, 15034, 12048, 20739, 15993, 7666, 4696, 19591, 13467, 9604, 23765]),
       list([14016, 16625, 15688, 26704, 18180, 526, 22272, 17418, 21107, 20958])],
      dtype=object)

In [0]:
import torch

test_nonrandom_batch_generator = batches_generator(
    batch_size=3,
    tokens_idxs=train_idxs[:7],
    tags_idxs=train_tag_idxs[:7],
    shuffle=False,
    allow_smaller_last_batch=True
)

batch_lengths = [3, 3, 1]
sequence_lengths = [26, 25, 8]
some_pad_tensor = torch.LongTensor([token2idx['<PAD>']] * 12)
some_outside_tensor = torch.LongTensor([tag2idx['O'] * 12])

for i, (tokens_batch, tags_batch) in enumerate(test_nonrandom_batch_generator):
    assert tokens_batch.dtype == torch.int64, 'tokens_batch is not LongTensor'
    assert tags_batch.dtype == torch.int64, 'tags_batch is not LongTensor'
    
    assert len(tokens_batch) == batch_lengths[i], 'wrong batch length'
    
    for one_token_sequence in tokens_batch:
        assert len(one_token_sequence) == sequence_lengths[i], 'wrong length of sequence in batch'
    
    if i == 0:
        assert torch.all(tokens_batch[2][-12:] == some_pad_tensor), "wrong padding"       
        assert torch.all(tags_batch[2][-12:] == some_outside_tensor), "wrong O tag"

## Часть 2. BiLSTM (3 балла)

Определите архитектуру сети, используя библиотеку pytorch. 

**Замечания:**
1. Для улучшения качества сети предлагается использовать дополнительный Embedding слой на входе (каждому слову ставится в соответствие обучаемый вектор). 

2. Не забудьте, что `<PAD>` токены не должны учавствовать в подсчёте функции потерь.

### help

Для тестирования сети мы подготовили для вас две функции:
 - *predict_tags*: получает батч данных и трансформирует его в список из токенов и предсказанных тегов;
 - *eval_conll*: вычисляет метрики precision, recall и F1

In [0]:
# from evaluation_ner import precision_recall_f1
from collections import OrderedDict

def _update_chunk(candidate, prev, current_tag, current_chunk, current_pos, prediction=False):
    if candidate == 'B-' + current_tag:
        if len(current_chunk) > 0 and len(current_chunk[-1]) == 1:
                current_chunk[-1].append(current_pos - 1)
        current_chunk.append([current_pos])
    elif candidate == 'I-' + current_tag:
        if prediction and (current_pos == 0 or current_pos > 0 and prev.split('-', 1)[-1] != current_tag):
            current_chunk.append([current_pos])
        if not prediction and (current_pos == 0 or current_pos > 0 and prev == 'O'):
            current_chunk.append([current_pos])
    elif current_pos > 0 and prev.split('-', 1)[-1] == current_tag:
        if len(current_chunk) > 0:
            current_chunk[-1].append(current_pos - 1)

def _update_last_chunk(current_chunk, current_pos):
    if len(current_chunk) > 0 and len(current_chunk[-1]) == 1:
        current_chunk[-1].append(current_pos - 1)

def _tag_precision_recall_f1(tp, fp, fn):
    precision, recall, f1 = 0, 0, 0
    if tp + fp > 0:
        precision = tp / (tp + fp) * 100
    if tp + fn > 0:
        recall = tp / (tp + fn) * 100
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, f1

def _aggregate_metrics(results, total_correct):
    total_true_entities = 0
    total_predicted_entities = 0
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    for tag, tag_metrics in results.items():
        n_pred = tag_metrics['n_predicted_entities']
        n_true = tag_metrics['n_true_entities']
        total_true_entities += n_true
        total_predicted_entities += n_pred
        total_precision += tag_metrics['precision'] * n_pred
        total_recall += tag_metrics['recall'] * n_true
    accuracy = total_correct / total_true_entities * 100
    total_precision = total_precision / total_predicted_entities if total_predicted_entities != 0 else 0
    total_recall = total_recall / total_true_entities
    if total_precision + total_recall > 0:
        if total_precision + total_recall >= 1e-16:
            total_f1 = 2 * total_precision * total_recall / (total_precision + total_recall)
        else:
            total_f1 = 0
    return total_true_entities, total_predicted_entities, \
           total_precision, total_recall, total_f1, accuracy

def _print_info(n_tokens, total_true_entities, total_predicted_entities, total_correct):
    print('processed {len} tokens ' \
          'with {tot_true} phrases; ' \
          'found: {tot_pred} phrases; ' \
          'correct: {tot_cor}.\n'.format(len=n_tokens,
                                         tot_true=total_true_entities,
                                         tot_pred=total_predicted_entities,
                                         tot_cor=total_correct))

def _print_metrics(accuracy, total_precision, total_recall, total_f1):
    print('precision:  {tot_prec:.2f}%; ' \
          'recall:  {tot_recall:.2f}%; ' \
          'F1:  {tot_f1:.2f}\n'.format(acc=accuracy,
                                           tot_prec=total_precision,
                                           tot_recall=total_recall,
                                           tot_f1=total_f1))

def _print_tag_metrics(tag, tag_results):
    print(('\t%12s' % tag) + ': precision:  {tot_prec:6.2f}%; ' \
                               'recall:  {tot_recall:6.2f}%; ' \
                               'F1:  {tot_f1:6.2f}; ' \
                               'predicted:  {tot_predicted:4d}\n'.format(tot_prec=tag_results['precision'],
                                                                         tot_recall=tag_results['recall'],
                                                                         tot_f1=tag_results['f1'],
                                                                         tot_predicted=tag_results['n_predicted_entities']))

def precision_recall_f1(y_true, y_pred, print_results=True, short_report=False):
    # Find all tags
    tags = sorted(set(tag[2:] for tag in y_true + y_pred if tag != 'O'))

    results = OrderedDict((tag, OrderedDict()) for tag in tags)
    n_tokens = len(y_true)
    total_correct = 0

    # For eval_conll_try we find all chunks in the ground truth and prediction
    # For each chunk we store starting and ending indices
    for tag in tags:
        true_chunk = list()
        predicted_chunk = list()
        for position in range(n_tokens):
            _update_chunk(y_true[position], y_true[position - 1], tag, true_chunk, position)
            _update_chunk(y_pred[position], y_pred[position - 1], tag, predicted_chunk, position, True)

        _update_last_chunk(true_chunk, position)
        _update_last_chunk(predicted_chunk, position)

        # Then we find all correctly classified intervals
        # True positive results
        tp = sum(chunk in predicted_chunk for chunk in true_chunk)
        total_correct += tp

        # And then just calculate errors of the first and second kind
        # False negative
        fn = len(true_chunk) - tp
        # False positive
        fp = len(predicted_chunk) - tp
        precision, recall, f1 = _tag_precision_recall_f1(tp, fp, fn)

        results[tag]['precision'] = precision
        results[tag]['recall'] = recall
        results[tag]['f1'] = f1
        results[tag]['n_predicted_entities'] = len(predicted_chunk)
        results[tag]['n_true_entities'] = len(true_chunk)

    total_true_entities, total_predicted_entities, \
           total_precision, total_recall, total_f1, accuracy = _aggregate_metrics(results, total_correct)

    if print_results:
        #_print_info(n_tokens, total_true_entities, total_predicted_entities, total_correct)
        _print_metrics(accuracy, total_precision, total_recall, total_f1)

        if not short_report:
            for tag, tag_results in results.items():
                _print_tag_metrics(tag, tag_results)
    return total_f1


In [0]:
def predict_tags(model, token_idxs_batch):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(token_idxs_batch)
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            if token_idx != token2idx['<PAD>']:
                tags.append(idx2tag[tag_idx])
                tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, x_batch)
        ground_truth_tags = [idx2tag[tag_idx] for tag_idx in y_batch[0]]

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(tags_batch[0] + ['O'])
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

### nn

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

In [0]:
class BiLSTMModel(torch.nn.Module):
    def __init__(self, vocabulary_size, n_tags, PAD_index,
                 embedding_dim, rnn_hidden_size,
                 dropout_zeroed_probability,
                 device='cpu'):
        '''
        Defines neural network structure.
        
        architecture: input -> Embedding -> BiLSTM -> Dropout -> Linear
        optimizer: Adam
        
        ----------
        Parameters
        
        vocabulary_size: int, number of words in vocabulary.
        n_tags: int, number of tags.
        PAD_index: int, index of padding character. Used for loss masking.
        embedding_dim: int, dimension of words' embeddings.
        rnn_hidden_size: int, number of hidden units in each LSTM cell
        dropout_zeroed_probability: float, dropout zeroed probability for Dropout layer.
        device: str, cpu or cuda:x
        '''
        super(BiLSTMModel, self).__init__()
        self.embeddng_dim = embedding_dim
        self.rnn_hidden_size = rnn_hidden_size
        
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim, padding_idx=PAD_index)
        
        self.lstm = nn.LSTM(input_size=self.embedding.embedding_dim,
                            hidden_size=rnn_hidden_size,
                            num_layers = 2, 
                            bidirectional=True,
                            batch_first=True)
        
        self.dropout = nn.Dropout(p=dropout_zeroed_probability)
        
        self.hidden2label = nn.Linear(rnn_hidden_size*2, n_tags)

        
    def forward(self, x_batch):
        '''
        Makes forward pass.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        x = self.embedding(x_batch.to('cuda'))
        lstm_out, _ = self.lstm(x)
        y = self.hidden2label(self.dropout(lstm_out))
        
        return y
    
    def predict_for_batch(self, x_batch):
        '''
        Returns predictions for x_batch.
        
        return type: torch.LongTensor
        return shape: (number of samples in batch, number words in sentence.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        y = self.forward(x_batch.to('cuda'))
        res = torch.argmax(y, dim=2)
        return res
    
    def train_on_batch(self, x_batch, y_batch, optimizer, loss_function, verbose=0):
        '''
        Trains model on the given batch.
        
        ----------
        Parameters
        x_batch: np.ndarray with shape (number of samples in batch, number words in sentence).
        y_batch: np.ndarray with shape (number of samples in batch).
        optimizer: torch.optimizer class
        loss_function: torch loss class
        '''
        optimizer.zero_grad()
        
        pred = self.forward(x_batch.to('cuda'))

        
        loss = 0
        for i in range(len(x_batch)):
          pr = pred[i]
          tr = y_batch[i].to('cuda')
          mask = x_batch[i] != 1
          loss += loss_function(pr,tr)
        
        if verbose:
          print('Loss : %.3f' % loss)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), .6)
        optimizer.step()

### Эксперименты

Задайте BiLSTMModel. Рекомендуемые параметры:
- *batch_size*: 32;
- начальное значение *learning_rate*: 0.01-0.001
- *dropout_zeroed_probability*: 0.7-0.9
- *embedding_dim*: 100-200
- *rnn_hidden_size*: 150-200

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1 мере на валидационной и тестовой выборках было не меньше 0.35. 

Если сеть плохо обучается, попробуйте использовать следующие модификации:
    * используйте gradient clipping
    * на каждой итерации уменьшайте learning rate (например, в 1.1 раз)
    * попробуйте вместо Adam другие оптимизаторы 
    * экспериментируйте с dropout

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

In [0]:
import numpy as np

In [18]:
model = BiLSTMModel(vocabulary_size = len(token2idx),
                    n_tags = len(tag2idx),
                    PAD_index = 1,
                    embedding_dim = 300,#100
                    rnn_hidden_size = 300,#180
                    dropout_zeroed_probability = 0.84#0.6
                    )

model.to('cuda')

BiLSTMModel(
  (embedding): Embedding(28821, 300, padding_idx=1)
  (lstm): LSTM(300, 300, num_layers=2, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.84, inplace=False)
  (hidden2label): Linear(in_features=600, out_features=23, bias=True)
)

In [0]:
train_idxs = [[token2idx[el] for el in sent] for sent in train_tokens]
train_tag_idxs = [[tag2idx[el] for el in sent] for sent in train_tags]

test_idxs = [[token2idx[el] for el in sent] for sent in test_tokens]
test_tag_idxs = [[tag2idx[el] for el in sent] for sent in test_tags]

eval_idxs = [[token2idx[el] for el in sent] for sent in validation_tokens]
eval_tag_idxs = [[tag2idx[el] for el in sent] for sent in validation_tags]

In [20]:
%%time
lr = 0.005#0.004
loss = nn.CrossEntropyLoss()
scores = []
for epoch in range(100):
    model.train()
    
    print('Epoch %s -------' % epoch)
    lr /= 1.1
    
    batch_gen = batches_generator(
        batch_size=32,
        tokens_idxs=train_idxs,
        tags_idxs=train_tag_idxs,
        shuffle=True,
        allow_smaller_last_batch=True
    )
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    it = 0
    for x_batch,y_batch in batch_gen:
        it += 1
        model.train_on_batch(x_batch,y_batch,optimizer, loss, verbose=(it%(60)==0))

    model.eval()
    if epoch % 1 == 0:
      train_idxs = [[token2idx[el] for el in sent] for sent in train_tokens]
      train_tag_idxs = [[tag2idx[el] for el in sent] for sent in train_tags]
      tmp = [epoch]
      print('eval:', end=' ')
      tmp.append(eval_conll(model, eval_idxs, eval_tag_idxs, short_report=True))
      print('test:', end=' ')
      tmp.append(eval_conll(model, test_idxs, test_tag_idxs, short_report=True))
      scores.append(tmp)

Epoch 0 -------
Loss : 10.359
Loss : 8.849
Loss : 9.829
eval: precision:  45.58%; recall:  12.50%; F1:  19.62

test: precision:  44.44%; recall:  11.94%; F1:  18.82

Epoch 1 -------
Loss : 6.213
Loss : 7.369
Loss : 5.957
eval: precision:  39.03%; recall:  25.56%; F1:  30.89

test: precision:  40.53%; recall:  25.21%; F1:  31.08

Epoch 2 -------
Loss : 1.548
Loss : 3.305
Loss : 2.061
eval: precision:  35.79%; recall:  31.72%; F1:  33.63

test: precision:  43.31%; recall:  29.52%; F1:  35.11

Epoch 3 -------
Loss : 2.821
Loss : 0.482
Loss : 1.116
eval: precision:  32.96%; recall:  32.84%; F1:  32.90

test: precision:  28.38%; recall:  31.67%; F1:  29.94

Epoch 4 -------
Loss : 1.134
Loss : 1.688
Loss : 0.620
eval: precision:  32.56%; recall:  33.96%; F1:  33.24

test: precision:  38.81%; recall:  33.67%; F1:  36.06

Epoch 5 -------
Loss : 0.282
Loss : 1.384
Loss : 1.336
eval: precision:  32.64%; recall:  35.07%; F1:  33.81

test: precision:  44.21%; recall:  34.83%; F1:  38.96

Epoch 6 -

KeyboardInterrupt: ignored

## Бонусная часть. Улучшение качества теггера (4 балла).

Улучшите качество теггера на данной задаче.

Бонусные баллы будут начисляться в зависимости от результата f1-меры (одновременно на тестовой и валидационной выборках!).

+ 1 балл — $> 0.38$
+ 2 балла — $> 0.4$
+ 3 балла — $> 0.425$
+ 4 балла — $> 0.45$

Разрешается использовать любые разумные способы (в том числе и не рассматривающиеся в курсе). Под неразумными способами понимаются любые, в которых используются модели, обученные на dev или test, а также модели, использующие утечки в данных, не относяющиеся к смыслу задачи.

In [0]:
######################################
######### YOUR CODE HERE #############
######################################        